In [ ]:
import BruinKeijman as bk
from datetime import datetime, timedelta
import os 
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import io
from nbformat import current
from scipy.stats import kendalltau
import seaborn as sns
from scipy import stats
%matplotlib inline

In [ ]:
sns.set(style='ticks')#, rc={'axes.facecolor':'black', 'figure.facecolor':'black'})

In [ ]:
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

In [ ]:
def crop(image, x1, x2, y1, y2):
    """
    Return the cropped image at the x1, x2, y1, y2 coordinates
    """
    if x2 == -1:
        x2=image.shape[1]-1
    if y2 == -1:
        y2=image.shape[0]-1

    mask = np.zeros(image.shape)
    mask[y1:y2+1, x1:x2+1]=1
    m = mask>0

    return image[m].reshape((y2+1-y1, x2+1-x1))

In [ ]:
def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

In [ ]:
def plot_colors_labels(ax, cb, im, im_title, cb_label, fg_color='white', bg_color='black'):    
    # IMSHOW    
    # set title plus title color
    ax.set_title(im_title, color=fg_color)
    
    # set figure facecolor
    ax.patch.set_facecolor(bg_color)
    
    # set tick and ticklabel color
    im.axes.tick_params(color=fg_color, labelcolor=fg_color)
    
    # set imshow outline
    for spine in im.axes.spines.values():
        spine.set_edgecolor(fg_color)    

    # COLORBAR
    # set colorbar label plus label color
    cb.set_label(cb_label, color=fg_color)
    
    # set colorbar tick color
    cb.ax.yaxis.set_tick_params(color=fg_color)
    
    # set colorbar edgecolor 
    cb.outline.set_edgecolor(fg_color)
    
    # set colorbar ticklabels
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=fg_color)

In [ ]:
def listall(RootFolder, varname='',extension='.png'):
    lists = [os.path.join(root, name)
             for root, dirs, files in os.walk(RootFolder)
             for name in files
             if varname in name
             if name.endswith(extension)]
    return lists

In [ ]:
def dictFromParam(rootDir):
    # input is the rootdirectory that contains the tif files
    # output is a dictionary with dates and absolute path to the items
    listDir = listall(rootDir, extension='.tif')

    # CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
    dates = []
    items = []
    for idx, item in enumerate(listDir):    
        base, file  = os.path.split(item)
        file, extension = os.path.splitext(file)
        date = datetime.strptime(file[-12:-4], '%Y%m%d')
        #print(date)
        dates.append(date)
        items.append(item)
    param_dict = dict(zip(dates, items))
    return param_dict

In [ ]:
root = r'D:\Projects\Pr\3492.10\ALBEDO\tif_IDW_100x100m'
listAlbedo = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_Albedo = []
items_Albedo = []
for idx, item in enumerate(listAlbedo):
    base, file  = os.path.split(listAlbedo[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_Albedo.append(date)
    items_Albedo.append(item)
dict_Albedo = dict(zip(dates_Albedo, items_Albedo))

In [ ]:
root = r'D:\Projects\Pr\3492.10\DIDSLF\tif_IDW_100x100m'
listInLW = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_InLW = []
items_InLW = []
for idx, item in enumerate(listInLW):
    base, file  = os.path.split(listInLW[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_InLW.append(date)
    items_InLW.append(item)
dict_InLW = dict(zip(dates_InLW, items_InLW))

In [ ]:
root = r'C:\Projects\Pr\3492.10\DIDSSF\tif_IDW_100x100m'
listInSW = listall(root, extension='.tif')

# CREATE DICTIONARY OF DATES AND ITEMS FOR ALBEDO FILES
dates_InSW = []
items_InSW = []
for idx, item in enumerate(listInSW):
    base, file  = os.path.split(listInSW[idx])
    file, extension = os.path.splitext(file)
    date = datetime.strptime(file[-12::], '%Y%m%d%H%M%S')
    #print(date)
    dates_InSW.append(date)
    items_InSW.append(item)
dict_InSW = dict(zip(dates_InSW, items_InSW))

In [ ]:
# CREATE DICTIONARIES OF DATES AND ITEMS FOR KNMI FILES
# Tmean
rootTmean = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Tmean'
dict_Tmean = dictFromParam(rootTmean)

# Tmin
rootTmin = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Tmin'
dict_Tmin = dictFromParam(rootTmin)

# Tmax
rootTmax = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Tmax'
dict_Tmax = dictFromParam(rootTmax)

# Kmean
rootKmean = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Kmean'
dict_Kmean = dictFromParam(rootKmean)

# Kmax
rootKmax = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Kmax'
dict_Kmax = dictFromParam(rootKmax)

# Kmin
rootKmin = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Kmin'
dict_Kmin = dictFromParam(rootKmin)

# Nmean
rootNmean = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Nmean'
dict_Nmean = dictFromParam(rootNmean)

# Pmean
rootPmean = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Pmean'
dict_Pmean = dictFromParam(rootPmean)

# Qsum
rootQsum = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Qsum'
dict_Qsum = dictFromParam(rootQsum)

# Umean
rootUmean = r'D:\Projects\Pr\3492.10\KNMI\tif_IDW_100x100m\Umean'
dict_Umean = dictFromParam(rootUmean)

In [ ]:
# load water mask or in fact load land and outer-sea mask
file_water       = r'D:\Projects\Pr\3492.10\Water//Water_100x100m.tif'
array_water      = gdal.Open(file_water).ReadAsArray()

In [ ]:
plt.imshow(array_water)
print ('value to mask is:', array_water[0][0] )

In [ ]:
font = {'family': 'arial',
        'color':  'white',
        'weight': 'normal',
        'size': 16,
        }

In [ ]:
# plot all data
for date in dates_InSW[0:1]:
    
    print ('date_All',date)    
    date_Albedo = min(dates_Albedo, key=lambda d: abs(d - date))
    print ('date_Albedo',date_Albedo)
    
    # get file names of each parameter
    file_Albedo  = dict_Albedo[date_Albedo]
    file_InLW    = dict_InLW[date]    
    file_InSW    = dict_InSW[date]
    file_Tmean   = dict_Tmean[date]
    file_Tmin    = dict_Tmin[date]
    file_Tmax    = dict_Tmax[date]
    file_Kmean   = dict_Kmean[date]
    file_Kmin    = dict_Kmin[date]
    file_Kmax    = dict_Kmax[date]
    file_Nmean   = dict_Nmean[date]    
    file_Pmean   = dict_Pmean[date]    
    file_Qsum    = dict_Qsum[date]    
    file_Umean   = dict_Umean[date] 
    
    # get arrays of each parameter
    array_Albedo = gdal.Open(file_Albedo).ReadAsArray()
    array_Albedo = np.ma.masked_where(array_water==255,array_Albedo)

    array_InLW   = gdal.Open(file_InLW).ReadAsArray()
    array_InLW   = np.ma.masked_where(array_water==255,array_InLW)  

    array_InSW   = gdal.Open(file_InSW).ReadAsArray()
    array_InSW   = np.ma.masked_where(array_water==255,array_InSW)      
    
    array_Tmean  = gdal.Open(file_Tmean).ReadAsArray()
    array_Tmean  = np.ma.masked_where(array_water==255,array_Tmean)
    
    array_Tmax   = gdal.Open(file_Tmax).ReadAsArray()
    array_Tmax   = np.ma.masked_where(array_water==255,array_Tmax)
    
    array_Tmin   = gdal.Open(file_Tmin).ReadAsArray()
    array_Tmin   = np.ma.masked_where(array_water==255,array_Tmin)
    
    array_Kmean  = gdal.Open(file_Kmean).ReadAsArray()
    array_Kmean  = np.ma.masked_where(array_water==255,array_Kmean)
    
    array_Kmax   = gdal.Open(file_Kmax).ReadAsArray()
    array_Kmax   = np.ma.masked_where(array_water==255,array_Kmax)
    
    array_Kmin   = gdal.Open(file_Kmin).ReadAsArray()
    array_Kmin   = np.ma.masked_where(array_water==255,array_Kmin)
    
    array_Nmean  = gdal.Open(file_Nmean).ReadAsArray()
    array_Nmean  = np.ma.masked_where(array_water==255,array_Nmean)
    
    array_Pmean  = gdal.Open(file_Pmean).ReadAsArray()    
    array_Pmean  = np.ma.masked_where(array_water==255,array_Pmean)
    
    array_Qsum   = gdal.Open(file_Qsum).ReadAsArray()
    array_Qsum   = np.ma.masked_where(array_water==255,array_Qsum)
    
    array_Umean  = gdal.Open(file_Umean).ReadAsArray()    
    array_Umean  = np.ma.masked_where(array_water==255,array_Umean)

#     #fig = plt.figure(figsize=(20,10))
#     fig, [[ax1,ax2,ax3,ax4],[ax5,ax6,ax7,ax8],[ax9,ax10,ax11,ax12]] = plt.subplots(3,4, figsize=(19,11))
#     fig.patch.set_facecolor('black')
#     fig.text(0.5, 1.02, date.strftime('%Y-%m-%d'), fontdict=font)    

#     im = ax1.imshow(np.ma.masked_where(array_Albedo == -999, array_Albedo), vmin=0, vmax=0.25)
#     cb = fig.colorbar(im,ax=ax1, format='%.2f')
#     plot_colors_labels(ax1, cb, im, 'Albedo', '-')

#     im = ax2.imshow(array_Tmean, vmin=2, vmax=27)
#     cb = plt.colorbar(im, ax=ax2, format='%.0f')    
#     plot_colors_labels(ax2, cb, im, 'Gemiddelde dagtemperatuur', '°C')    

#     im = ax3.imshow(array_Tmax, vmin=2, vmax=27)
#     cb = plt.colorbar(im, ax=ax3, format='%.0f')
#     plot_colors_labels(ax3, cb, im, 'Maximum dagtemperatuur', '°C')    

#     im = ax4.imshow(array_Tmin, vmin=2, vmax=27)
#     cb = plt.colorbar(im, ax=ax4, format='%.0f')
#     plot_colors_labels(ax4, cb, im, 'Minimum dagtemperatuur', '°C')

#     im = ax5.imshow(array_Kmean, vmin=2+273.15, vmax=27+273.15)
#     cb = plt.colorbar(im, ax=ax5, format='%.0f')
#     plot_colors_labels(ax5, cb, im, 'Gemiddelde dagtemperatuur', 'K')    

#     im = ax6.imshow(array_Kmax, vmin=2+273.15, vmax=27+273.15)
#     cb = plt.colorbar(im, ax=ax6, format='%.0f')    
#     plot_colors_labels(ax6, cb, im, 'Maximum dagtemperatuur', 'K')        
    
# #     im = ax7.imshow(array_Kmin, vmin=2+273.15, vmax=27+273.15)
# #     cb = plt.colorbar(im, ax=ax7, format='%.2f')
# #     plot_colors_labels(ax7, cb, im, 'Minimum dagtemperatuur', 'K')

#     im = ax7.imshow(array_InSW, vmin=30, vmax=350)
#     cb = plt.colorbar(im, ax=ax7, format='%.0f')
#     plot_colors_labels(ax7, cb, im, 'Inkomende kortgolvige straling', 'W/m2')    

#     im = ax8.imshow(array_Nmean, vmin=0, vmax=1)
#     cb = plt.colorbar(im, ax=ax8, format='%.1f')
#     plot_colors_labels(ax8, cb, im, 'Bedekkingsgraad bewolking', '-')

#     im = ax9.imshow(array_Pmean, vmin=998.5, vmax=1026)
#     cb = plt.colorbar(im, ax=ax9, format='%.0f')    
#     plot_colors_labels(ax9, cb, im, 'Luchtdruk', 'kPa')    

#     im = ax10.imshow(array_Qsum, vmin=30, vmax=350)
#     cb = plt.colorbar(im, ax=ax10, format='%.0f')
#     plot_colors_labels(ax10, cb, im, 'Dagsom globale straling', 'W/m2')

#     im = ax11.imshow(array_Umean,vmin=0.6,vmax=1.0)
#     cb = plt.colorbar(im, ax=ax11, format='%.2f')    
#     plot_colors_labels(ax11, cb, im, 'Relatieve vochtigheid', '-')    

#     im = ax12.imshow(array_InLW,vmin=272,vmax=380)
#     cb = plt.colorbar(im, ax=ax12, format='%.0f')    
#     plot_colors_labels(ax12, cb, im, 'Inkomende langgolvige straling', 'W/m2')
    
#     #fig.patch.set_facecolor('black') 
#     plt.tight_layout()
    
#     file_out = r'C:\Projects\Pr\3492.10\BruinKeijman\png//spBLACKv2_'+date.strftime('%Y%m%d')+'.png'    
#     fig.savefig(file_out, facecolor=fig.get_facecolor(), dpi=200, edgecolor='none', bbox_inches='tight')
#     #plt.show()
#     plt.clf()
#     plt.cla()
#     plt.close(fig)
#     plt.close()
    
    

In [ ]:
# # plot only difference between InShortwave and Globale dagsom
# for date in dates_InSW:
    
#     print ('date_All',date)    
#     date_Albedo = min(dates_Albedo, key=lambda d: abs(d - date))
#     print ('date_Albedo',date_Albedo)
    
#     # get file names of each parameter  
#     file_InSW    = dict_InSW[date]   
#     file_Qsum    = dict_Qsum[date]     
    
#     array_InSW   = gdal.Open(file_InSW).ReadAsArray()
#     array_InSW   = np.ma.masked_where(array_water==255,array_InSW)      
    
#     array_Qsum   = gdal.Open(file_Qsum).ReadAsArray()
#     array_Qsum   = np.ma.masked_where(array_water==255,array_Qsum)
    
#     array_difInSW_Qsum = array_InSW - array_Qsum
    
#     # plot the figure
#     fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(15,4))
#     fig.patch.set_facecolor('black')
#     fig.text(0.5, 1.02, date.strftime('%Y-%m-%d'), fontdict=font)

#     im = ax1.imshow(array_InSW, vmin=30, vmax=350)
#     cb = plt.colorbar(im, ax=ax1, format='%.0f')
#     plot_colors_labels(ax1, cb, im, 'Inkomende kortgolvige straling', 'W/m2')    

#     im = ax2.imshow(array_Qsum, vmin=30, vmax=350)
#     cb = plt.colorbar(im, ax=ax2, format='%.0f')
#     plot_colors_labels(ax2, cb, im, 'Dagsom globale straling', 'W/m2')
    
#     im = ax3.imshow(array_difInSW_Qsum, cmap='BrBG',vmin=-50, vmax=50)
#     cb = plt.colorbar(im, ax=ax3, format='%.0f')
#     plot_colors_labels(ax3, cb, im, 'Verschil inkomend - dagsom', 'W/m2')

#     plt.tight_layout()
    
#     file_out = r'C:\Projects\Pr\3492.10\BruinKeijman\png//SW_Q_'+date.strftime('%Y%m%d')+'.png'    
#     fig.savefig(file_out, facecolor=fig.get_facecolor(), dpi=200, edgecolor='none', bbox_inches='tight')
# #    plt.show()
#     plt.clf()
#     plt.cla()
#     plt.close(fig)
#     plt.close()

In [ ]:
x = date.timetuple().tm_yday
print ('doy: ',x)

In [ ]:
execute_notebook('2017-01-25 De Bruin-Keijmanmethode.ipynb')

In [ ]:
# plot all data
for date in dates_InSW:
    
    print ('date_All',date)    
    date_Albedo = min(dates_Albedo, key=lambda d: abs(d - date))
    print ('date_Albedo',date_Albedo)
    
    # get file names of each parameter
    file_Albedo  = dict_Albedo[date_Albedo]
    file_InLW    = dict_InLW[date]    
    file_InSW    = dict_InSW[date]
    file_Tmean   = dict_Tmean[date]
    file_Tmin    = dict_Tmin[date]
    file_Tmax    = dict_Tmax[date]
    file_Kmean   = dict_Kmean[date]
    file_Kmin    = dict_Kmin[date]
    file_Kmax    = dict_Kmax[date]
    file_Nmean   = dict_Nmean[date]    
    file_Pmean   = dict_Pmean[date]    
    file_Qsum    = dict_Qsum[date]    
    file_Umean   = dict_Umean[date] 
    
    # LOADING DATASETS :: get arrays of each parameter
#     array_Albedo = gdal.Open(file_Albedo).ReadAsArray()
#     array_Albedo = np.ma.masked_where(array_water==255,array_Albedo)

    array_InLW   = gdal.Open(file_InLW).ReadAsArray()
    array_InLW   = np.ma.masked_where(array_water==255,array_InLW)  

    array_InSW   = gdal.Open(file_InSW).ReadAsArray()
    array_InSW   = np.ma.masked_where(array_water==255,array_InSW)      
    
#     array_Tmean  = gdal.Open(file_Tmean).ReadAsArray()
#     array_Tmean  = np.ma.masked_where(array_water==255,array_Tmean)
    
    array_Tmax   = gdal.Open(file_Tmax).ReadAsArray()
    array_Tmax   = np.ma.masked_where(array_water==255,array_Tmax)
    
    array_Tmin   = gdal.Open(file_Tmin).ReadAsArray()
    array_Tmin   = np.ma.masked_where(array_water==255,array_Tmin)
    
    array_Kmean  = gdal.Open(file_Kmean).ReadAsArray()
    array_Kmean  = np.ma.masked_where(array_water==255,array_Kmean)
    
    array_Kmax   = gdal.Open(file_Kmax).ReadAsArray()
    array_Kmax   = np.ma.masked_where(array_water==255,array_Kmax)
    
    array_Kmin   = gdal.Open(file_Kmin).ReadAsArray()
    array_Kmin   = np.ma.masked_where(array_water==255,array_Kmin)
    
    array_Nmean  = gdal.Open(file_Nmean).ReadAsArray()
    array_Nmean  = np.ma.masked_where(array_water==255,array_Nmean)
    
    array_Pmean  = gdal.Open(file_Pmean).ReadAsArray()    
    array_Pmean  = np.ma.masked_where(array_water==255,array_Pmean)
    
    array_Qsum   = gdal.Open(file_Qsum).ReadAsArray()
    array_Qsum   = np.ma.masked_where(array_water==255,array_Qsum)
    
    array_Umean  = gdal.Open(file_Umean).ReadAsArray()    
    array_Umean  = np.ma.masked_where(array_water==255,array_Umean)

    
    # Berekening L-netto based on incoming longwave
    # Stefan-Boltzmann constant = 5.670367e-08 W m^-2 K^-4
    L_netto_InLW = bk_L_netto_InLW(epsilon=0.96,sigma=constants.Stefan_Boltzmann, T_a_max=array_Kmax, T_a_min=array_Kmin, L_inLW=array_InLW)
    
    # L-netto based on station data estimates
    # Berekening e_sat
    e_sat = bk_e_s(array_Tmin, array_Tmax)

    # Berekening e_z
    e_z = bk_e_z(array_Umean, e_sat)

    # Berekening N_rel
    N_rel = bk_N_rel(C=array_Nmean)

    # Berekening L-netto 
    L_netto_station = bk_L_netto_station(epsilon=0.96, sigma=constants.Stefan_Boltzmann, T_a_max=array_Kmax, T_a_min=array_Kmin, e_z=e_z, N_rel=N_rel)

    # Lnetto difference Incoming Longwave - Station based Lnetto
    L_netto_dif = L_netto_InLW - L_netto_station    
    
    # crop to province Friesland
    x1,x2,y1,y2 =  1980,2470, 1150, 1580
    L_netto_InLW_FR    = crop(L_netto_InLW,    x1,x2,y1,y2)
    L_netto_station_FR = crop(L_netto_station, x1,x2,y1,y2)
    L_netto_dif_FR     = crop(L_netto_dif,     x1,x2,y1,y2)
    
    
    # plot the figure
    fig, [[ax1,ax2,ax3],[ax4,ax5,ax6]] = plt.subplots(2,3, figsize=(15,8))
    fig.patch.set_facecolor('black')
    fig.text(0.5, 1.02, date.strftime('%Y-%m-%d'), fontdict=font)

    im = ax1.imshow(L_netto_InLW, vmin=0, vmax=50, cmap='viridis')
    cb = plt.colorbar(im, ax=ax1, format='%.0f')
    plot_colors_labels(ax1, cb, im, 'Netto langgolvige straling (InLW)', 'W/m2')    

    im = ax2.imshow(L_netto_station, vmin=0, vmax=50, cmap='viridis')
    cb = plt.colorbar(im, ax=ax2, format='%.0f')
    plot_colors_labels(ax2, cb, im, 'Netto langgolvige straling (stations)', 'W/m2')

    im = ax3.imshow(L_netto_dif, cmap='BrBG',vmin=-50, vmax=50)
    cb = plt.colorbar(im, ax=ax3, format='%.0f')
    plot_colors_labels(ax3, cb, im, 'Verschil Lnetto_InLW - Lnetto_stations', 'W/m2')

    im = ax4.imshow(L_netto_InLW_FR, vmin=0, vmax=50, cmap='viridis')
    cb = plt.colorbar(im, ax=ax4, format='%.0f')
    plot_colors_labels(ax4, cb, im, 'FR:Netto langgolvige straling (InLW)', 'W/m2')    

    im = ax5.imshow(L_netto_station_FR, vmin=0, vmax=50, cmap='viridis')
    cb = plt.colorbar(im, ax=ax5, format='%.0f')
    plot_colors_labels(ax5, cb, im, 'FR:Netto langgolvige straling (stations)', 'W/m2')

    im = ax6.imshow(L_netto_dif_FR, cmap='BrBG',vmin=-50, vmax=50)
    cb = plt.colorbar(im, ax=ax6, format='%.0f')
    plot_colors_labels(ax6, cb, im, 'FR:Verschil Lnetto_InLW - Lnetto_stations', 'W/m2')


    plt.tight_layout()
    
    file_out = r'C:\Projects\Pr\3492.10\BruinKeijman\png//Lnetto_'+date.strftime('%Y%m%d')+'.png'    
    fig.savefig(file_out, facecolor=fig.get_facecolor(), dpi=200, edgecolor='none', bbox_inches='tight')
#     plt.show()
    plt.clf()
    plt.cla()
    plt.close(fig)
    plt.close()

    x = L_netto_InLW.reshape(-1).compressed()
    y = L_netto_station.reshape(-1).compressed()

    g = sns.jointplot(x, y, kind="hex", size=6, gridsize=30, stat_func=r2)
    g.set_axis_labels("Lnetto (OLW:station estimate ILW:EUMETSAT)", "Lnetto (OLW,ILW:EUMETSAT)")
    g.fig.suptitle('Netherlands')
    g.fig.text(0.75, 0.95,date.strftime('%Y-%m-%d'))
    file_out_plt_NL = r'C:\Projects\Pr\3492.10\BruinKeijman\png//Lnetto_scatterNL_'+date.strftime('%Y%m%d')+'.png'    
    g.savefig(file_out_plt_NL, dpi=200)

    x_FR = L_netto_InLW_FR.reshape(-1).compressed()
    y_FR = L_netto_station_FR.reshape(-1).compressed()

    g = sns.jointplot(x_FR, y_FR, kind="hex", size=6, gridsize=30, stat_func=r2)#,shade=True, shade_lowest=False)
    g.set_axis_labels("Lnetto (W/m2) - OLW:station estimate ILW:EUMETSAT", "Lnetto (W/m2) -  OLW,ILW:station estimate")
    g.fig.suptitle('Friesland province')
    g.fig.text(0.75, 0.95,date.strftime('%Y-%m-%d'))
    file_out_plt_FR = r'C:\Projects\Pr\3492.10\BruinKeijman\png//Lnetto_scatterFR_'+date.strftime('%Y%m%d')+'.png'    
    g.savefig(file_out_plt_FR, dpi=200)

    plt.clf()
    plt.cla()
    plt.close(fig)
    plt.close(g.fig)
    plt.close()

In [ ]:
# Berekening L-netto based on incoming longwave
# Stefan-Boltzmann constant = 5.670367e-08 W m^-2 K^-4
L_netto_InLW = bk_L_netto_InLW(epsilon=0.96,sigma=constants.Stefan_Boltzmann, T_a_max=array_Kmax, T_a_min=array_Kmin, L_inLW=array_InLW)

In [ ]:
# L-netto based on station data estimates
# Berekening e_sat
e_sat = bk_e_s(array_Tmin, array_Tmax)

# Berekening e_z
e_z = bk_e_z(array_Umean, e_sat)

# Berekening N_rel
N_rel = bk_N_rel(C=array_Nmean)

# Berekening L-netto 
L_netto_station = bk_L_netto_station(epsilon=0.96, sigma=constants.Stefan_Boltzmann, T_a_max=array_Kmax, T_a_min=array_Kmin, e_z=e_z, N_rel=N_rel)


In [ ]:
L_netto_dif = L_netto_InLW - L_netto_station

In [ ]:
# crop to province Friesland
x1,x2,y1,y2 =  1980,2470, 1150, 1580
L_netto_InLW_FR    = crop(L_netto_InLW,    x1,x2,y1,y2)
L_netto_station_FR = crop(L_netto_station, x1,x2,y1,y2)
L_netto_dif_FR     = crop(L_netto_dif,     x1,x2,y1,y2)

In [ ]:
# # plot the figure
# fig, [[ax1,ax2,ax3],[ax4,ax5,ax6]] = plt.subplots(2,3, figsize=(15,8))
# fig.patch.set_facecolor('black')
# fig.text(0.5, 1.02, date.strftime('%Y-%m-%d'), fontdict=font)

# im = ax1.imshow(L_netto_InLW, vmin=0, vmax=50, cmap='viridis')
# cb = plt.colorbar(im, ax=ax1, format='%.0f')
# plot_colors_labels(ax1, cb, im, 'Netto langgolvige straling (InLW)', 'W/m2')    

# im = ax2.imshow(L_netto_station, vmin=0, vmax=50, cmap='viridis')
# cb = plt.colorbar(im, ax=ax2, format='%.0f')
# plot_colors_labels(ax2, cb, im, 'Netto langgolvige straling (stations)', 'W/m2')

# im = ax3.imshow(L_netto_dif, cmap='BrBG',vmin=-50, vmax=50)
# cb = plt.colorbar(im, ax=ax3, format='%.0f')
# plot_colors_labels(ax3, cb, im, 'Verschil Lnetto_InLW - Lnetto_stations', 'W/m2')

# im = ax4.imshow(L_netto_InLW_FR, vmin=0, vmax=50, cmap='viridis')
# cb = plt.colorbar(im, ax=ax4, format='%.0f')
# plot_colors_labels(ax4, cb, im, 'FR:Netto langgolvige straling (InLW)', 'W/m2')    

# im = ax5.imshow(L_netto_station_FR, vmin=0, vmax=50, cmap='viridis')
# cb = plt.colorbar(im, ax=ax5, format='%.0f')
# plot_colors_labels(ax5, cb, im, 'FR:Netto langgolvige straling (stations)', 'W/m2')

# im = ax6.imshow(L_netto_dif_FR, cmap='BrBG',vmin=-50, vmax=50)
# cb = plt.colorbar(im, ax=ax6, format='%.0f')
# plot_colors_labels(ax6, cb, im, 'FR:Verschil Lnetto_InLW - Lnetto_stations', 'W/m2')


# plt.tight_layout()

In [ ]:
# x = L_netto_InLW.reshape(-1).compressed()
# y = L_netto_station.reshape(-1).compressed()

# g = sns.jointplot(x, y, kind="hex", size=6, gridsize=30, stat_func=r2)
# g.set_axis_labels("Lnetto (OLW:station estimate ILW:EUMETSAT)", "Lnetto (OLW,ILW:EUMETSAT)")
# g.fig.suptitle('Netherlands')
# file_out_plt_NL = r'C:\Projects\Pr\3492.10\BruinKeijman\png//Lnetto_scatterNL_'+date.strftime('%Y%m%d')+'.png'    
# #g.savefig(file_out_plt_NL, dpi=200)

# x_FR = L_netto_InLW_FR.reshape(-1).compressed()
# y_FR = L_netto_station_FR.reshape(-1).compressed()

g = sns.jointplot(x_FR, y_FR, kind="hex", size=6, gridsize=30, stat_func=r2)#,shade=True, shade_lowest=False)
g.set_axis_labels("Lnetto (W/m2) - OLW:station estimate ILW:EUMETSAT", "Lnetto (W/m2) -  OLW,ILW:station estimate")
g.fig.suptitle('Friesland province')
g.fig.text(0.75, 0.95,date.strftime('%Y-%m-%d'))
file_out_plt_FR = r'C:\Projects\Pr\3492.10\BruinKeijman\png//Lnetto_scatterFR_'+date.strftime('%Y%m%d')+'.png'    
#g.savefig(file_out_plt_FR, dpi=200)


In [ ]:
plt.scatter(x_FR,y_FR)

In [ ]:
x_FR.shape

In [ ]:
# Berekening γ
λ = bk_lamda(array_Tmean)
γ = bk_gamma(array_Pmean, λ)

In [ ]:
# Berekening s
e_sat = bk_e_s(array_Tmin, array_Tmax)
s = bk_s(e_sat, array_Kmean)

In [ ]:
# im = plt.imshow(s)
# plt.colorbar(im)

In [ ]:
bk_T_vwt()
G = bk_G()